# Data Collection and Preprocessing

* Attribute data: 2017–2022 American Community Survey (ACS) 5-year estimates at the census tract level for the City of Riverside, California
* Geometry data: polygons for census tracts from Census bureau's TIGER/Line Shapefiles

In [1]:
import json
import requests
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import geopandas as gpd
import seaborn as sns
import numpy as np

## Use Census API to query attributes of interest from 2017-2022 ACS 5-year estimates

You will need to request a U.S. Census Data API Key in order to use the Census API to access the ACS data. Sign up for an API key using this [link](https://api.census.gov/data/key_signup.html).

In [2]:
key_api = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX" #your requested Census Data API Key (should be a string type)
url_B = "https://api.census.gov/data/2022/acs/acs5?get=%s&for=tract:*&in=state:06&in=county:065,071&key="+ key_api

In [3]:
var_numer_ownerNoM = "B25140_006E"
var_remove_ownerNoM = "B25140_009E"
var_deno30_ownerNoM = "B25140_007E"

var_numer_ownerM = "B25140_002E"
var_remove_ownerM = "B25140_005E"
var_deno30_ownerM = "B25140_003E"
var_numer_renter = "B25140_010E"
var_remove_renter = "B25140_013E"
var_deno30_renter = "B25140_011E"

vars_all=[var_numer_ownerNoM,var_remove_ownerNoM,var_deno30_ownerNoM,
                   var_numer_ownerM, var_remove_ownerM,var_deno30_ownerM,
                   var_numer_renter,var_remove_renter,var_deno30_renter]
var_all = ",".join(vars_all)
r = requests.get(url_B%var_all)
content = r.json()
df_ACS_emp = pd.DataFrame(np.asarray(content[1:]), columns=content[0])
df_ACS_emp[vars_all] = df_ACS_emp[vars_all].astype(float)
numer = df_ACS_emp[var_numer_ownerNoM]-df_ACS_emp[var_remove_ownerNoM]+df_ACS_emp[var_numer_ownerM]-df_ACS_emp[var_remove_ownerM]+df_ACS_emp[var_numer_renter]-df_ACS_emp[var_remove_renter]
df_ACS_emp["AllC"] = numer.astype(int)
#df_ACS_emp = df_ACS_emp.rename(columns = {var_numer:"Emp2Pop"})
df_ACS_emp["All30C"] = (df_ACS_emp[var_deno30_ownerNoM]+df_ACS_emp[var_deno30_ownerM]+df_ACS_emp[var_deno30_renter]).astype(int)

df_ACS_emp["All30P"] = (df_ACS_emp["All30C"]/numer).round(3)

df_ACS_emp["GEOID"] = df_ACS_emp.state+df_ACS_emp.county+df_ACS_emp.tract
df = df_ACS_emp.drop(columns=["state","county","tract"]+vars_all).set_index("GEOID")

#list_df.append(df)
df.head()

,AllC,All30C,All30P
GEOID,,,
06065030101,299,142,0.475
06065030103,572,218,0.381
06065030104,1803,691,0.383
06065030200,1650,489,0.296
06065030300,1577,713,0.452


### Obtain geometries (polygons) for census tracts and merge them with attribute data

In [4]:
url = "https://www2.census.gov/geo/tiger/TIGER2022/TRACT/tl_2022_06_tract.zip" # all US tracts
gdf_CA = gpd.read_file(url) 
gdf_county = gdf_CA[gdf_CA.COUNTYFP.isin(["065"])].set_index("GEOID") #filter tracts in Riverside county
gdf_county = gdf_county[["geometry"]].merge(df,right_index=True, left_index=True)

url_place = "https://www2.census.gov/geo/tiger/TIGER2022/PLACE/tl_2022_06_place.zip" # all US places
gdf_place = gpd.read_file(url_place)
gdf_place = gdf_place[gdf_place.GEOID=="0662000"] #boundary of Riverside city

# conduct spatial join to obtain all tracts in Riverside county) that intersect with the boundary of Riverside city
gdf_Riverside =  gdf_county.sjoin(gdf_place[["geometry"]]).drop(columns="index_right")

# save obtained tracts to a shapefile 
gdf_Riverside.to_file("data/CostBurden_Riverside.shp")